# Procesamiento de Datos de Seguridad

Este notebook sirve para probar transformaciones y validaciones antes de implementarlas en el script de procesamiento.

## Objetivos:
1. Transformar datos raw a formato tidy
2. Validar calidad de datos
3. Generar datasets listos para análisis

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Configuración
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Rutas
PROJECT_ROOT = Path.cwd().parent
DATA_RAW = PROJECT_ROOT / 'data' / 'raw'
DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
DATA_INTERIM = PROJECT_ROOT / 'data' / 'interim'

# Crear directorios
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
DATA_INTERIM.mkdir(parents=True, exist_ok=True)

## 1. Procesamiento: Percepción de Inseguridad

In [ ]:
# Cargar datos raw
df_percepcion_raw = pd.read_csv(DATA_RAW / 'indicador_inseguridad_estados.csv')
print(f"Datos cargados: {df_percepcion_raw.shape}")
df_percepcion_raw.head()

In [ ]:
# Inspeccionar estructura
df_percepcion_raw.info()

In [ ]:
# Renombrar solo columna de clave (mantener 'año' con ñ)
df_perc = df_percepcion_raw.copy()
df_perc = df_perc.rename(columns={
    'clave': 'cve_entidad'
})

print("Columnas renombradas:")
df_perc.columns.tolist()

In [ ]:
# Convertir tipos de datos
df_perc['año'] = df_perc['año'].astype(int)
df_perc['valor'] = pd.to_numeric(df_perc['valor'], errors='coerce')
df_perc['cve_entidad'] = df_perc['cve_entidad'].astype(str).str.zfill(2)

df_perc.info()

In [ ]:
# Agregar columna de categoría de percepción
df_perc['nivel_percepcion'] = pd.cut(
    df_perc['valor'],
    bins=[0, 50000, 70000, 85000, 100000],
    labels=['Bajo', 'Medio', 'Alto', 'Muy Alto'],
    include_lowest=True
)

# Indicador de nacional
df_perc['es_nacional'] = df_perc['entidad'] == 'Nacional'

df_perc.head(10)

In [ ]:
# Verificar distribución de niveles
print("Distribución por nivel de percepción:")
print(df_perc['nivel_percepcion'].value_counts())
print(f"\nPorcentaje:")
print(df_perc['nivel_percepcion'].value_counts(normalize=True) * 100)

### Validaciones de Calidad - Percepción

In [ ]:
# 1. Verificar valores nulos
print("Valores nulos:")
print(df_perc.isnull().sum())
print(f"\nPorcentaje de nulos:")
print(df_perc.isnull().sum() / len(df_perc) * 100)

In [ ]:
# 2. Verificar duplicados
duplicados = df_perc.duplicated(subset=['año', 'cve_entidad'], keep=False)
print(f"Registros duplicados: {duplicados.sum()}")

if duplicados.sum() > 0:
    print("\nDuplicados encontrados:")
    print(df_perc[duplicados].sort_values(['año', 'cve_entidad']))

In [ ]:
# 3. Verificar rango de valores
print(f"Rango de valores:")
print(f"  Mínimo: {df_perc['valor'].min()}")
print(f"  Máximo: {df_perc['valor'].max()}")
print(f"  Media: {df_perc['valor'].mean():.2f}")
print(f"  Mediana: {df_perc['valor'].median():.2f}")

# Detectar valores fuera de rango esperado [0, 100000]
fuera_rango = df_perc[(df_perc['valor'] < 0) | (df_perc['valor'] > 100000)]
print(f"\nValores fuera de rango [0, 100000]: {len(fuera_rango)}")

In [ ]:
# 4. Verificar completitud temporal
años_esperados = set(range(2011, 2026))
print(f"Años esperados: {len(años_esperados)}")
print(f"Rango: {min(años_esperados)} - {max(años_esperados)}\n")

print("Completitud por entidad:")
for entidad in sorted(df_perc['entidad'].unique()):
    df_ent = df_perc[df_perc['entidad'] == entidad]
    años_disponibles = set(df_ent['año'].unique())
    pct_completo = len(años_disponibles) / len(años_esperados) * 100
    
    status = "✓" if pct_completo == 100 else "⚠"
    print(f"{status} {entidad:25s}: {pct_completo:5.1f}% ({len(años_disponibles)}/{len(años_esperados)})")

In [ ]:
# Ordenar y guardar (prueba)
df_perc_final = df_perc.sort_values(['cve_entidad', 'año']).reset_index(drop=True)

print(f"\nDataset final:")
print(f"  Registros: {len(df_perc_final)}")
print(f"  Columnas: {df_perc_final.columns.tolist()}")

df_perc_final.head()

## 2. Procesamiento: Incidencia Delictiva

Este dataset puede tener estructura más compleja

In [ ]:
# Cargar datos raw
df_delictiva_raw = pd.read_csv(
    DATA_RAW / 'incidencia_delictiva_estatal_2015_2025.csv',
    encoding='latin-1'
)

print(f"Datos cargados: {df_delictiva_raw.shape}")
print(f"\nColumnas ({len(df_delictiva_raw.columns)}):")
for i, col in enumerate(df_delictiva_raw.columns, 1):
    print(f"  {i}. {col}")

In [ ]:
# Ver primeras filas
df_delictiva_raw.head()

In [ ]:
# Normalizar nombres de columnas
df_delic = df_delictiva_raw.copy()
df_delic.columns = df_delic.columns.str.strip().str.lower().str.replace(' ', '_')

print("Columnas normalizadas:")
for col in df_delic.columns[:10]:
    print(f"  - {col}")

In [ ]:
# Identificar columnas clave
print("Análisis de columnas:")
print(f"\nColumnas numéricas: {len(df_delic.select_dtypes(include=[np.number]).columns)}")
print(f"Columnas de texto: {len(df_delic.select_dtypes(include=['object']).columns)}")

# Buscar columnas de tiempo
cols_tiempo = [col for col in df_delic.columns if any(x in col for x in ['año', 'ano', 'mes', 'fecha'])]
print(f"\nColumnas de tiempo: {cols_tiempo}")

# Buscar columnas de ubicación
cols_ubicacion = [col for col in df_delic.columns if any(x in col for x in ['entidad', 'estado', 'municipio'])]
print(f"Columnas de ubicación: {cols_ubicacion}")

In [ ]:
# Información detallada
df_delic.info()

In [ ]:
# Valores únicos en columnas de texto
for col in df_delic.select_dtypes(include=['object']).columns[:5]:
    print(f"\n{col}:")
    print(f"  Valores únicos: {df_delic[col].nunique()}")
    if df_delic[col].nunique() < 20:
        print(f"  Valores: {df_delic[col].unique()[:10]}")

### Guardar Versiones Procesadas

In [ ]:
# Guardar percepción procesada
output_perc = DATA_PROCESSED / 'percepcion_inseguridad_procesado.csv'
df_perc_final.to_csv(output_perc, index=False)
print(f"✓ Guardado: {output_perc}")

# Guardar solo estados (sin nacional)
df_estados = df_perc_final[df_perc_final['entidad'] != 'Nacional'].copy()
output_estados = DATA_PROCESSED / 'percepcion_inseguridad_estados.csv'
df_estados.to_csv(output_estados, index=False)
print(f"✓ Guardado: {output_estados}")

In [ ]:
# Guardar incidencia delictiva (versión completa en interim)
output_delic = DATA_INTERIM / 'incidencia_delictiva_completa.csv'
df_delic.to_csv(output_delic, index=False)
print(f"✓ Guardado: {output_delic}")

## 3. Resumen del Procesamiento

In [ ]:
print("=" * 80)
print("RESUMEN DEL PROCESAMIENTO")
print("=" * 80)

print("\n📊 PERCEPCIÓN DE INSEGURIDAD:")
print(f"  Total de registros: {len(df_perc_final)}")
print(f"  Período: {df_perc_final['año'].min()} - {df_perc_final['año'].max()}")
print(f"  Entidades: {df_perc_final['entidad'].nunique()}")
print(f"  Columnas: {', '.join(df_perc_final.columns)}")

print("\n📊 INCIDENCIA DELICTIVA:")
print(f"  Total de registros: {len(df_delic)}")
print(f"  Columnas: {len(df_delic.columns)}")

print("\n✅ ARCHIVOS GENERADOS:")
print(f"  • {output_perc.name}")
print(f"  • {output_estados.name}")
print(f"  • {output_delic.name}")